In [136]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [148]:
data = pd.read_csv('data/california_fires.csv')
data = data.sort_values('DISCOVERY_DATE')
labels = pd.DataFrame()
labels['FIRE_SIZE'] = data['FIRE_SIZE']
labels['FIRE_SIZE_CLASS'] = data['FIRE_SIZE_CLASS']


/Users/sharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [149]:
data = data.drop(columns=[ 'OBJECTID', 'SOURCE_SYSTEM_TYPE', 'Shape', 'FIRE_NAME', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'SOURCE_REPORTING_UNIT', 'STAT_CAUSE_CODE', 'FIPS_NAME', 'DISCOVERY_TIME', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'COUNTY', 'Unnamed: 0', 'FIRE_SIZE_CLASS', 'FIRE_SIZE']).reset_index()
data.head()

,index,SOURCE_SYSTEM,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE
0,34267,DOI-WFMI,1992,2448622.5,1,Debris Burning,33.166700,-116.634200,2.0,BIA,CA
1,135389,ST-CACDF,1992,2448622.5,1,Children,33.663889,-116.171944,14.0,MISSING/NOT SPECIFIED,CA
2,10455,FS-FIRESTAT,1992,2448622.5,1,Lightning,38.205000,-120.335000,13.0,STATE OR PRIVATE,CA
3,135391,ST-CACDF,1992,2448623.5,2,Children,33.678056,-116.171944,14.0,MISSING/NOT SPECIFIED,CA
4,135390,ST-CACDF,1992,2448623.5,2,Miscellaneous,33.896111,-116.990000,14.0,MISSING/NOT SPECIFIED,CA


In [144]:
dummies = pd.get_dummies(data)
dummies = dummies.drop(columns = 'index')

In [140]:
from sklearn.feature_selection import SelectKBest, chi2, RFE, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from itertools import compress

In [141]:
model = RandomForestClassifier()
rfe = RFE(model, 10)
rfe.fit(dummies, labels.FIRE_SIZE_CLASS)

RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
  n_features_to_select=10, step=1, verbose=0)

In [142]:
list(compress(dummies.columns.values, rfe.get_support()))

['index',
 'FIRE_YEAR',
 'DISCOVERY_DATE',
 'DISCOVERY_DOY',
 'LATITUDE',
 'LONGITUDE',
 'OWNER_CODE',
 'STAT_CAUSE_DESCR_Equipment Use',
 'STAT_CAUSE_DESCR_Miscellaneous',
 'OWNER_DESCR_MISSING/NOT SPECIFIED']

In [155]:
dummy_data = data[['FIRE_YEAR',
 'DISCOVERY_DATE',
 'DISCOVERY_DOY',
 'OWNER_CODE',
 'STAT_CAUSE_DESCR',
 'OWNER_DESCR']]
dummies = pd.get_dummies(dummy_data)
dummies.head()


,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,OWNER_CODE,STAT_CAUSE_DESCR_Arson,STAT_CAUSE_DESCR_Campfire,STAT_CAUSE_DESCR_Children,STAT_CAUSE_DESCR_Debris Burning,STAT_CAUSE_DESCR_Equipment Use,STAT_CAUSE_DESCR_Fireworks,STAT_CAUSE_DESCR_Lightning,STAT_CAUSE_DESCR_Miscellaneous,STAT_CAUSE_DESCR_Missing/Undefined,STAT_CAUSE_DESCR_Powerline,STAT_CAUSE_DESCR_Railroad,STAT_CAUSE_DESCR_Smoking,STAT_CAUSE_DESCR_Structure,OWNER_DESCR_BIA,OWNER_DESCR_BLM,OWNER_DESCR_BOR,OWNER_DESCR_COUNTY,OWNER_DESCR_FOREIGN,OWNER_DESCR_FWS,OWNER_DESCR_MISSING/NOT SPECIFIED,OWNER_DESCR_MUNICIPAL/LOCAL,OWNER_DESCR_NPS,OWNER_DESCR_OTHER FEDERAL,OWNER_DESCR_PRIVATE,OWNER_DESCR_STATE,OWNER_DESCR_STATE OR PRIVATE,OWNER_DESCR_TRIBAL,OWNER_DESCR_UNDEFINED FEDERAL,OWNER_DESCR_USFS
0,1992,2448622.5,1,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1992,2448622.5,1,14.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1992,2448622.5,1,13.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1992,2448623.5,2,14.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1992,2448623.5,2,14.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
dummies.to_csv('dummies_data.csv')